In [1]:
# Import packages for ...
## pulling data from XML and HTML files
from bs4 import BeautifulSoup 

## automating web browser interaction
from selenium import webdriver # module containing implementations of browser drivers
from webdriver_manager.chrome import ChromeDriverManager # Chrome driver 
from selenium.webdriver.support import expected_conditions as EC # method for writing code that waits until conditions are met 
from selenium.webdriver.support.ui import WebDriverWait # method for writing code that implements implicit or explicit waits
from selenium.webdriver.common.by import By # method for locating elements by their attributes
from selenium.webdriver import ActionChains # module for implementing browser interactions 

## data manipulation
import pandas as pd
from datetime import datetime
import numpy as np
import time
import os

## plotting
import matplotlib.pyplot as plt

In [2]:
# Open up a chrome page
driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://ida.ussc.gov/analytics/saw.dll?Dashboard"
driver.get(url)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/kate/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


## Import .py function

In [283]:
from sentencing_outcomes import *

In [282]:
# Reload the script
import sys, importlib
importlib.reload(sys.modules['sentencing_outcomes'])

<module 'sentencing_outcomes' from '/Users/kate/Documents/Data Science/dashboard-web-scraping/sentencing_outcomes.py'>

## Sentence Length

In [7]:
# Navigate to Sentencing Outcomes, Sentence Length page
nav_to_sentencingoutcomes(driver, "Sentence Length")

In [94]:
sentence_length_all = pd.DataFrame() # Instantiate a df
sentence_length_all['Race'] = ""
sentence_length_all['Crime Type'] = ""

In [250]:
r = race[0] # spin through all - manually for now
#ct = crime_type[3] # spin through all
one_checkbox(driver, r, "true", "Race", val=0) # toggle 

Element is a checkbox
Checkbox updated: Other


In [263]:
for ct in crime_type: # spin through all
    one_checkbox(driver, ct, "true", "Crime Type", val=0) # toggle
    ## Append the data to DF
    sentence_length_all = sentence_length_all.append(sentence_length(driver, r, ct), ignore_index=True)
    one_checkbox(driver, ct, "None", "Crime Type", val=0) # toggle
one_checkbox(driver, r, "None", "Race", val=0) # toggle 

Element is a checkbox
Checkbox updated: Tax
['Distribution of Sentence Length 60 to 119 Months', 'Distribution of Sentence Length 24 to 59 Months', 'Distribution of Sentence Length Up to 24 Months', 'Distribution of Imprisonment Length 60 to 119 Months', 'Distribution of Imprisonment Length 24 to 59 Months', 'Distribution of Imprisonment Length Up to 24 Months', 'Sentence Length (Average Months)', 'Sentence Length (Median Months)', 'Imprisonment Length (Average Months)', 'Imprisonment Length (Median Months)', 'Supervised Release (Average Months)', 'Supervised Release (Median Months)']


['2.2%', '15.2%', '82.6%', '3.4%', '20.7%', '75.9%', '15', '12', '21', '16', '25', '36']
DF has been generated for race Other, crime type Tax
Element is a checkbox
Checkbox updated: Tax
Element is a checkbox
Checkbox updated: Other
['Distribution of Sentence Length 60 to 119 Months', 'Distribution of Sentence Length 24 to 59 Months', 'Distribution of Sentence Length Up to 24 Months', 'Distribution of Im

In [ ]:
sentence_length_all

In [265]:
sentence_length_all.to_csv("sentence_length.csv")

## Plea Status

In [268]:
# Navigate to Sentencing Outcomes, Sentence Length page
nav_to_sentencingoutcomes(driver, "Plea Status")

In [288]:
def all_plea_status(race):
    plea_status_all = pd.DataFrame()
    for i in race:
        print(i)
        one_checkbox(driver, i, "true", "Race")
        time.sleep(5)
        expand_list(driver)
        plea_status_all = plea_status_all.append(plea_status(driver, i), ignore_index=True)
        time.sleep(5)
        one_checkbox(driver, i, "None", "Race")
    return(plea_status_all)

In [289]:
plea_status_all = all_plea_status(race)
plea_status_all

White
Element is a checkbox
Checkbox status as expected: White


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//img[contains(@src,'/analytics/res/v-*xNdJt5L9yA/s_blafp/viewui/pivot/showallrows_ena.png')]"}
  (Session info: chrome=84.0.4147.125)


In [ ]:
plea_status_all.to_csv('Plea Status All.csv')

## Sentence Type

In [ ]:
# Navigate to Sentencing Outcomes, Sentence Length page
nav_to_sentencingoutcomes(driver, "Sentence Type")

In [286]:
def all_sentencing_type(race):
    plea_status_all = pd.DataFrame()
    for i in race:
        one_checkbox(driver, i, "true", "Race")
        time.sleep(5)
        expand_list()
        time.sleep(5)
        sentencing_type_all = sentencing_type_all.append(sentence_type(driver, i), ignore_index=True)
        time.sleep(5)
        one_checkbox(driver, i, "true", "Race")
        time.sleep(5)
    return(sentencing_type_all)


In [287]:
race = ['White','Black','Hispanic', 'Other']
sentencing_type_all = all_sentencing_type(race)
sentencing_type_all

NameError: name 'tick_checkbox' is not defined

In [ ]:
sentence_type_all = sentence_type_all.append(sentence_type("Other"), ignore_index=True)

In [ ]:
sentence_type_all